In [71]:
from stsci.tools import capable
capable.OF_GRAPHICS = False

from pyraf import iraf
from pyraf.iraf import gemini, gemtools, gmos, onedspec
import fileSelect as fs
import copy, os
from astropy.io import fits


In [72]:
import pyds9

name = pyds9.ds9_targets()
if name != 'None':
    name1 = name[0].split()[1]
    #print(name1)
    d = pyds9.DS9(name1)

In [73]:
pwd

u'/home/mmarcano/Documents/GeminiReduceTTU/Object7'

First specified the directory where all the data was downloaded and extracted. Then defined the sqlite3 database. This contains the information of all the files. This obsLog.sqlite3 file was created by the python script obslog.py in the raw data directory. The script will create a database with the fields and metadata

In [74]:
datadirall = '../raw/'
databasename = datadirall+'obsLog.sqlite3'
dbFile= datadirall+'obsLog.sqlite3'

 From the work_directory:
 Create the query dictionary of essential parameter=value pairs.

In [75]:
qd = {'Full':{'use_me':1,
       'Instrument':'GMOS-S','CcdBin':'2 2','RoI':'Full',
       'Disperser':'R400+_G5325','CentWave':646.0,'AperMask':'0.5arcsec',
       'Object':'Obj-7',
       'DateObs':'2018-09-15:2018-09-30'}
      }


 In the tutorial there are two regions but here only one and it is full. Just changed the regions list to only Full. It should work. Bias only depend on:
 instrument Roi and CcdBin according to fileSelect file
 
     use_me=1 AND ObsType='BIAS' AND ObsClass LIKE '%Cal'
    AND Instrument=:Instrument AND RoI=:RoI AND CcdBin=:CcdBin

In [76]:
gemtools.gemextn.unlearn()    # Disarm a bug in gbias
gmos.gbias.unlearn()
biasFlags = {
    'logfile':'biasLog.txt','rawpath':'../raw/','fl_vardq':'yes','verbose':'no'
}
regions = ['Full']
for r in regions:
    # The following SQL generates the list of full-frame files to process.
    SQL = fs.createQuery('bias', qd[r])
    biasFiles = fs.fileListQuery(dbFile, SQL, qd[r])

    # The str.join() funciton is needed to transform a python list into a
    # comma-separated string of file names that IRAF can understand.
    if len(biasFiles) > 1:
        gmos.gbias(','.join(str(x) for x in biasFiles), 'MCbias'+r,
                   **biasFlags)

ERROR - GBIAS: Output file MCbiasFull.fits already exists.


In [77]:
#Clean up after BIAs

iraf.imdel('gS2018*.fits')

In [78]:
# Open it in DS9 to see it
biasfit = fits.open('MCbiasFull.fits')
d.set_fits(biasfit)

1

# Flat Field

In [79]:
# Set the task parameters.
qd['Full'].update({'DateObs':'*'})
gmos.gireduce.unlearn()
gmos.gsflat.unlearn()
# The response fitting should be done interactively.
flatFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_dark':'no',
    'fl_fixpix':'no','fl_oversize':'no','fl_vardq':'yes','fl_fulldq':'yes',
    'rawpath':'../raw/','fl_inter':'no','fl_detec':'yes',
    'function':'spline3','order':'13,11,28',
    'logfile':'gsflatLog.txt','verbose':'no'
    }
for r in regions:
    qr = qd[r]
    flatFiles = fs.fileListQuery(dbFile, fs.createQuery('gcalFlat', qr), qr)
    if len(flatFiles) > 0:
        gmos.gsflat (','.join(str(x) for x in flatFiles), 'MCflat'+r,
                bias='MCbias'+r, **flatFlags)


ERROR - GSFLAT: Output file MCflatFull already exists.
ERROR - GSFLAT: 1 fatal errors found.
 
ERROR - GSFLAT: Program execution failed with 1 errors.


In [80]:
iraf.imdel('gS2018*.fits,gsS2018*.fits')

In [81]:
flatfile = fits.open('MCflatFull.fits')
d.set_fits(flatfile)

1

# Basic Reduction

In [82]:
# Set task parameters.
gmos.gsreduce.unlearn()
sciFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_gscrrej':'no',
    'fl_dark':'no','fl_flat':'yes','fl_gmosaic':'yes','fl_fixpix':'no',
    'fl_gsappwave':'yes','fl_oversize':'no',
    'fl_vardq':'yes','fl_fulldq':'yes','rawpath':'../raw/',
    'fl_inter':'no','logfile':'gsreduceLog.txt','verbose':'no'
}
arcFlags = copy.deepcopy(sciFlags)
arcFlags.update({'fl_flat':'no','fl_vardq':'no','fl_fulldq':'no'})


# Arc exposures
for r in regions:
    qr = qd[r]
    arcFiles = fs.fileListQuery(dbFile, fs.createQuery('arc', qr), qr)
    if len(arcFiles) > 0:
        gmos.gsreduce (','.join(str(x) for x in arcFiles), bias='MCbias'+r,
                  **arcFlags)


# Science exposures
r = 'Full'
sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qd[r]), qd[r])
if len(sciFiles) > 0:
    gmos.gsreduce (','.join(str(x) for x in sciFiles), bias='MCbias'+r,
              flatim='MCflat'+r, **sciFlags)

GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged


In [83]:
iraf.imdel('gS2018*.fits')

# Wavelength Calibration (Check this later)

# Need to select specific wavecals to match science exposures.

**Image rectification and wavelength linearization depend upon the wavelength calibration, using the arc lamp exposures taken immediately before each sequence of science and standard star exposures** (see Wavelength Calibration). In this case, the default medium-resolution line list will work well. The fit to the dispersion relation should be performed interactively, but for expediency we will use a previously determined functional fit. 


I had tpo manually put the name of the two arc Files. I am not sure how may to use. 


- [ ] On that day there is only one arc


Science ones are:

- S20181004S0022.fits 
- S20181004S0022.fits 

at 23:53 and 23:55

The closet arc is on 2018-10-04 called :S20181004S0209.fits

## But his depends on CentWave

for other one do:



have same name as science and have write center wavelegnth



In [85]:
# Set task parameters
gmos.gswavelength.unlearn()
waveFlags = {
    'coordlist':'gmos$data/CuAr_GMOS.dat','fwidth':6,'nsum':50,
    'function':'chebyshev','order':5,
    'fl_inter':'no','logfile':'gswaveLog.txt','verbose':'yes'
    }
# Must select specific wavecals to match science exposures.
#prefix = 'gsS20180708S00'
prefix = 'gsS20180710S0020'

#for arc in [arcname+'.fits']:
for arc in ['']:
     #gmos.gswavelength (prefix+arc, **waveFlags)
    gmos.gswavelength (prefix+arc, **waveFlags)        

--------------------------------------------------------------------------------
GSWAVELENGTH -- Fri Jan 18 11:22:02 CST 2019
 
inimages = gsS20180710S0020
Calibrating extension: 1

AUTOIDENTIFY: NOAO/IRAF V2.16 mmarcano@tux Fri 11:22:02 18-Jan-2019
  Spectrum                 # Found   Midpoint Dispersion        RMS
  gsS20180710S0020_001[*,1044]         89    6463.91       -1.5      0.271

REIDENTIFY: NOAO/IRAF V2.16 mmarcano@tux Fri 11:22:02 18-Jan-2019
  Reference image = gsS20180710S0020_001, New image = gsS20180710S0020_001, Refit = yes
          Image Data    Found     Fit Pix Shift  User Shift  Z Shift      RMS
gsS20180710S0020_001[*,1034] 87/89   73/87     0.0188     -0.0269  -5.6E-6    0.262
gsS20180710S0020_001[*,1024] 86/87   72/89    -0.0498      0.0727  1.49E-5    0.249
gsS20180710S0020_001[*,1014] 88/89   75/90   -0.00943      0.0143  1.94E-6    0.265
gsS20180710S0020_001[*,1004] 90/90   79/91     0.0297     -0.0422  -1.1E-5    0.463
gsS20180710S0020_001[*,994] 91/91   76

# Advanced Processing


In the example tutorial:

>The targets in this program were observed in 3 slit orientations, and a few exposures were obtained at each position. This provides an opportunity to combine the sequential exposures at each position to remove cosmic rays, rather than rejecting CRs on single frames using the gsreduce.fl_gscrrej+ flag or running the gemcrspec task. The combined exposures for each target are then wavelength calibrated, and sky subtracted. First set the processing parameters.

Here we have two exposures of the same thing. Supposed it is the same. 

In [86]:
# Set task parameters.
gemtools.gemcombine.unlearn()
sciCombFlags = {
    'combine':'average','reject':'ccdclip',
    'fl_vardq':'yes','fl_dqprop':'yes',
    'logfile':'gemcombineLog.txt.txt','verbose':'no'
}
stdCombFlags = copy.deepcopy(sciCombFlags)
stdCombFlags.update({'fl_vardq':'no','fl_dqprop':'no'})
gmos.gstransform.unlearn()
transFlags = {
    'fl_vardq':'yes','interptype':'linear','fl_flux':'yes',
    'logfile':'gstransLog.txt'
}
# The sky regions should be selected with care, using e.g. prows/pcols:
#   pcols ("tAM2306b.fits[SCI]", 1100, 2040, wy1=40, wy2=320)
gmos.gsskysub.unlearn()
skyFlags = {
    'fl_oversize':'no','fl_vardq':'yes','logfile':'gsskysubLog.txt'
}

# The sky regions should be selected with care, using e.g. prows/pcols:

# Science Targets

##  Use a dictionary to associate science targets with Arcs and sky regions.
have to put the bad colum in *list one* to fix an also manually change the arc and sky postion to substract.

In [94]:
sciTargets = {
    'Obj-7':{'arc':'gsS20180710S0020','sky':'100:700'},
}

prefix = "gs"

for targ,p in sciTargets.iteritems():
    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    sciOut = targ.split('-')[0]+targ[-1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    gemtools.gemcombine (','.join(prefix+str(x) for x in sciFiles),
                         sciOut, **sciCombFlags)
    gmos.gstransform(sciOut, wavtraname=p['arc'], **transFlags)
    gmos.gsskysub('t'+sciOut, long_sample=p['sky'], **skyFlags)

ERROR - GEMCOMBINE: Output file Obj7 already exists.
-------------------------------------------------------------------------------
GSTRANSFORM -- Fri Jan 18 19:05:40 CST 2019
 

inimages   = Obj7
outimages  = 
outprefix  = t
fl_stran   = no
fl_wavtran = yes
wavtraname = gsS20180710S0020
database   = database
fl_vardq   = yes
interptype = linear
lambda1    = INDEF
lambda2    = INDEF
dx         = INDEF
nx         = INDEF
lambdalog  = no
ylog       = no
fl_flux    = yes
gratingdb  = gmos$data/GMOSgratings.dat
filterdb   = gmos$data/GMOSfilters.dat
key_dispaxis = DISPAXIS
dispaxis   = 1
sci_ext    = SCI
var_ext = VAR
dq_ext  = DQ

Transforming Obj7.fits[SCI,1]
MDF row: 1

NOAO/IRAF V2.16 mmarcano@tux Fri 19:05:41 18-Jan-2019
  Transform Obj7.fits[SCI,1] to tmpsci22806_2675.
  Conserve flux per pixel.
  User coordinate transformations:
    gsS20180710S0020_001
  Interpolation is linear.
  Using edge extension for out of bounds pixel values.
  Output coordinate parameters are:
    x1 =    

In [93]:

gmos.gstransform ('e'+scienceobect, wavtraname=p['arc'], **transFlags)
gmos.gsskysub ('t'+'e'+scienceobect, long_sample=p['sky'], **skyFlags)

-------------------------------------------------------------------------------
GSTRANSFORM -- Fri Jan 18 19:04:01 CST 2019
 
ERROR - GSTRANSFORM: Input image eObj7.fits does not exist or is not MEF
ERROR - GSTRANSFORM: 1 errors found with input parameters. Exiting.

GSTRANSFORM exit status: error
-------------------------------------------------------------------------------

----------------------------------------------------------------------------
GSSKYSUB -- Fri Jan 18 19:04:01 CST 2019

input        = teObj7.fits
output       = 
outpref      = s
fl_vardq     = yes
fl_oversize  = no
sci_ext      = SCI
var_ext      = VAR
dq_ext       = DQ
long_sample  = 100:700
mos_sample   = 0.9
mosobjsize   = 1.0
naverage     = 1
function     = chebyshev
order        = 1
low_reject   = 2.5
high_reject  = 2.5
niterate     = 2
grow         = 0.0
fl_inter     = no

ERROR - GSSKYSUB: Input image teObj7.fits does not exist
ERROR - GSSKYSUB: 1 error(s) found. Exiting.

GSSKYSUB done. Exit status bad. 

In [87]:
sciTargets = {
    'Obj-7':{'arc':'gsS20180710S0020','sky':'100:700'},
}

for targ,p in sciTargets.iteritems():

    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    sciOut = targ.split('-')[0]+targ[-1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    
    
    #interpolate
    scienceobect = sciOut+'.fits'
    
    a = fits.open(sciOut+'.fits')
    axis1 = a[1].header['NAXIS1']
    axis2 = a[1].header['NAXIS2']


    #listone=[[1900 , 1970, 1, 2088],[2370,2395, 1, 2088],[300,320, 1, 2088]]
    listone=[[2822 , 2826, 1, 2088],[1172,1180,1,2088],[2370,2400,1,2088]]

    # x1 x2 y1 y2
    filemaskbad = 'maskbadcol.txt'
    filemaskiraf = 'maskbadcol.pl'
    with open(filemaskbad,'w') as f:
        for l in listone:
            x1,x2,y1,y2 = l
            f.write('{} {} {} {}\n'.format(x1,x2,y1,y2))
    iraf.text2mask(filemaskbad,filemaskiraf,axis1,axis2)

    iraf.copy(scienceobect,'tempstObj.fits')
    iraf.proto.fixpix('tempstObj.fits[SCI,1]',filemaskiraf,linterp='1,2,3,4',verbose='yes',pixel='no')
    iraf.copy('tempstObj.fits','e'+scienceobect)
    
    iraf.imarith('maskbadcol.pl', '+', 'e'+scienceobect+'[dq,1]', \
                 'tmpdq'+'e'+scienceobect)
    iraf.imcopy('tmpdq'+'e'+scienceobect+'[*,*]','e'+scienceobect+'[dq,1][*,*]' )
    os.remove('tempstObj.fits')
    os.remove('tmpdq'+'e'+scienceobect)


    
    gmos.gstransform ('e'+scienceobect, wavtraname=p['arc'], **transFlags)
    gmos.gsskysub ('t'+'e'+scienceobect, long_sample=p['sky'], **skyFlags)


IOError: [Errno 2] No such file or directory: 'Obj7.fits'

In [ ]:
iraf.imdel("gsS2018*.fits")

After sky substract fix column again

In [14]:
def fixcol(file,listcols):
    """Carefi; woth naxis. Should chedk with header is SCI"""
    a = fits.open(file)
    axis1 = a[2].header['NAXIS1']
    axis2 = a[2].header['NAXIS2']
    
    scienceobect = file


    #listone=[[1900 , 1970, 1, 2088],[2370,2395, 1, 2088],[300,320, 1, 2088]]
    listone=[[2370,2400,1,2088]]

    # x1 x2 y1 y2
    filemaskbad = 'maskbadcol.txt'
    filemaskiraf = 'maskbadcol.pl'
    with open(filemaskbad,'w') as f:
        for l in listone:
            x1,x2,y1,y2 = l
            f.write('{} {} {} {}\n'.format(x1,x2,y1,y2))
    iraf.text2mask(filemaskbad,filemaskiraf,axis1,axis2)

    iraf.copy(scienceobect,'tempstObj.fits')
    iraf.proto.fixpix('tempstObj.fits[SCI,1]',filemaskiraf,linterp='1,2,3,4',verbose='yes',pixel='no')
    iraf.copy('tempstObj.fits','e'+scienceobect)
    
    iraf.imarith('maskbadcol.pl', '+', 'e'+scienceobect+'[dq,1]', \
                 'tmpdq'+'e'+scienceobect)
    iraf.imcopy('tmpdq'+'e'+scienceobect+'[*,*]','e'+scienceobect+'[dq,1][*,*]' )
    os.remove('tempstObj.fits')
    os.remove('tmpdq'+'e'+scienceobect)

In [82]:
fixcol('steObj6.fits',[[2370,2400,1,2088]])

FIXPIX: image tempstObj.fits[SCI,1] with mask maskbadcol.pl
tmpdqesteObj6.fits[*,*] -> esteObj6.fits[dq,1][*,*]


## Try

In [71]:
onedspec.nsum=2
onedspec.sarith('esteObj6.fits[SCI]', 'copy', '', 'esteObj6.ms',
                  apertures='1037-1060x2')


pcols tObj5.fits[SCI] 1500 1700 wy1=40 

prows tAM2306b.fits[SCI] 520 720 wcs='world'



Sky http://www.gemini.edu/sciops/telescopes-and-sites/observing-condition-constraints/optical-sky-background